# Вийнятки в Python

In [1]:
print(1 / 0)

ZeroDivisionError: division by zero

In [ ]:
import sys


def wc(filename):
    count = 0
    with open(filename) as f:
        for line in f:
            count += 1

    return count


def process_file(filename):
    count = wc(filename)
    print(f'file {filename} has {count} lines')


def _main():
    process_file(sys.argv[1])


if __name__ == "__main__":
    _main()


In [3]:
class MyClass:
    pass

obj = MyClass()
obj.foo

AttributeError: 'MyClass' object has no attribute 'foo'

In [4]:
d = {'foo': 1}
d['bar']

KeyError: 'bar'

In [5]:
l = [1, 2]
l[10]

IndexError: list index out of range

In [6]:
int('asdf')

ValueError: invalid literal for int() with base 10: 'asdf'

In [7]:
1 + '10'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

## Обробка виключень

In [8]:
try:
    1 / 0
except:
    print('Zero Division')

Zero Division


In [10]:
while True:
    try:
        raw = int(input('Enter a number:\t'))
        number = raw
        break
    except:
        print('Value error')

Enter a number:	bbb
Value error
Enter a number:	q
Value error
Enter a number:	100066600001


### Обробка кількох виключень

In [11]:
while True:
    try:
        number = int(input('Enter a number:\t'))
        break
    except ValueError:
        print('Value error')
    except KeyboardInterrupt:
        print('exit')

Enter a number:	
Value error
exit
Enter a number:	7


In [13]:
total = 100_000

while True:
    try:
        number = int(input('Enter a number:\t'))
        total /= number
        print(total)
        break
    except (ValueError, ZeroDivisionError):
        print('Value error')
    except KeyboardInterrupt:
        print('Interrupt by user')

Enter a number:	b
Value error
Interrupt by user
Enter a number:	0
Value error
Enter a number:	2
50000.0


In [15]:
# +-- LookupError
#     +-- IndexError
#     +-- KeyError

print(issubclass(IndexError, LookupError),
     issubclass(KeyError, LookupError))

True True


### наслідування

In [19]:
database = {
    'red': ['fox', 'flower'],
    'green': ['pease', 'M', 'python']
}

try:
    color = input('enter the color: \t')
    number = int(input('enter the order number: \t'))
    
    label = database[color][number]
    print('You choose:', label)
except LookupError:
    print('Diin\'t find the item')

enter the color: 	green
enter the order number: 	3
Diin't find the item


## Блок `finally`

In [22]:
f = open('/etc/hosts')

try:
    for line in f:
        print(line.rstrip('\n'))
        1/0
        
except OSError:
    print('Відмовлено в доступі')
finally:
    # Закриваємо файл у будь-якому випадку:
    f.close()

##


ZeroDivisionError: division by zero